# Reviewing db stuff

In [71]:
import os
import sys
import sqlite3
from sqlite3 import Error

In [72]:
db_name = 'pdf_texts.db'

In [73]:
conn = sqlite3.connect(db_name)
cursor = conn.cursor()
cursor.execute('SELECT * FROM pdf_texts')
rows = cursor.fetchall()
conn.close()

In [74]:
# get columns in database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()
cursor.execute('PRAGMA table_info(pdf_texts)')
columns = cursor.fetchall()
conn.close()
print(f"Database: {db_name}")
print(f"Number of rows: {len(rows)}")
print("Columns:")
for column in columns:
    print(f"  {column[1]} (Type: {column[2]})")


Database: pdf_texts.db
Number of rows: 2604
Columns:
  id (Type: INTEGER)
  text (Type: TEXT)
  url (Type: TEXT)
  timestamp (Type: TEXT)
  html_metadata (Type: TEXT)


In [76]:
# sample a column
if len(rows) > 0:
    print("\PDF URLs:")
    for row in rows[:-5]:  # Display first 5 rows
        print(f"  {row[2]}")  # Assuming the URL is in the second column (index 1)
if len(rows) == 0:
    print("No rows found in the database.")

\PDF URLs:
  https://supreme.justia.com/cases/federal/us/502/244/case.pdf
  https://supreme.justia.com/cases/federal/us/502/279/case.pdf
  https://supreme.justia.com/cases/federal/us/502/410/case.pdf
  https://supreme.justia.com/cases/federal/us/502/21/case.pdf
  https://supreme.justia.com/cases/federal/us/502/346/case.pdf
  https://supreme.justia.com/cases/federal/us/502/215/case.pdf
  https://supreme.justia.com/cases/federal/us/502/93/case.pdf
  https://supreme.justia.com/cases/federal/us/502/105/case.pdf
  https://supreme.justia.com/cases/federal/us/502/81/case.pdf
  https://supreme.justia.com/cases/federal/us/502/301/case.pdf
  https://supreme.justia.com/cases/federal/us/502/164/case.pdf
  https://supreme.justia.com/cases/federal/us/502/251/case.pdf
  https://supreme.justia.com/cases/federal/us/502/62/case.pdf
  https://supreme.justia.com/cases/federal/us/502/478/case.pdf
  https://supreme.justia.com/cases/federal/us/502/129/case.pdf
  https://supreme.justia.com/cases/federal/us/50

# REVIEW

In [77]:
## I want random snippets somewhere in the middle of each row of text
print("\nRandom snippets from the text column:")
for row in rows[:5]:  # Display first 5 rows
    text = row[4]
    if len(text) > 100:  # Ensure there's enough text to sample
        start = len(text) // 4  # Start sampling from the middle
        end = start + 100  # Get a snippet of 100 characters
        snippet = text[start:end]
        print(f"  Snippet: {snippet}...")  # Display the snippet
    else:
        print("  Text too short for snippet.")
        


Random snippets from the text column:
  Snippet: h-below-tablet item">
<strong>Argued:</strong>
<span>December 2, 1991</span>
</div>
<div class="flex...
  Snippet: h-below-tablet item">
<strong>Argued:</strong>
<span>October 7, 1991</span>
</div>
<div class="flex-...
  Snippet: h-below-tablet item">
<strong>Argued:</strong>
<span>October 15, 1991</span>
</div>
<div class="flex...
  Snippet: h-below-tablet item">
<strong>Argued:</strong>
<span>October 15, 1991</span>
</div>
<div class="flex...
  Snippet: h-below-tablet item">
<strong>Argued:</strong>
<span>November 5, 1991</span>
</div>
<div class="flex...


## Read as a pandas dataframe

In [78]:
import pandas as pd
conn = sqlite3.connect(db_name)
df = pd.read_sql_query("SELECT * FROM pdf_texts", conn)
conn.close()
print("\nDataFrame loaded successfully.")
print(df.head())



DataFrame loaded successfully.
   id                                               text  \
0   1  502us2$20Z 01-22-99 14:22:41 PAGES OPINPGT\n24...   
1   2  502us2$22M 08-19-96 17:40:23 PAGES OPINPGT\n27...   
2   3  502us2$27Z 01-22-99 08:37:00 PAGES OPINPGT\n41...   
3   4  502us1$$4Z 08-21-96 15:22:03 PAGES OPINPGT\n21...   
4   5  502us2$25Z 01-22-99 08:28:07 PAGES OPINPGT\n34...   

                                                 url  \
0  https://supreme.justia.com/cases/federal/us/50...   
1  https://supreme.justia.com/cases/federal/us/50...   
2  https://supreme.justia.com/cases/federal/us/50...   
3  https://supreme.justia.com/cases/federal/us/50...   
4  https://supreme.justia.com/cases/federal/us/50...   

                    timestamp  \
0  2025-06-25T23:40:05.507904   
1  2025-06-25T23:40:10.965825   
2  2025-06-25T23:40:16.437248   
3  2025-06-25T23:40:21.753877   
4  2025-06-25T23:40:27.145292   

                                       html_metadata  
0  <div class="f

In [79]:
for txt in df['text'][100].split('\n'):
    print(f"{txt}")  # Display the text from the first row

505us2109K 07-09-96 20:05:24 PAGES OPINPGT
557 OCTOBER TERM, 1991
Syllabus
CITY OF BURLINGTON v.DAGUE et al.
certiorari to the united states court of appeals for
the second circuit
No. 91±810. Argued April 21, 1992ÐDecided June 24, 1992
After ruling on the merits for respondents, the District Court determined
that they were ªsubstantially prevailingº parties entitled to ªreason-ableº attorney's fees under the attorney's fee provisions of the SolidWaste Disposal Act and the Clean Water Act. The District Court cal-culated the fee award by, inter alia, enhancing the ªlodestarº amount
by 25% on the grounds that respondents' attorneys were retained on acontingent-fee basis and that without such enhancement respondentswould have faced substantial dif®culties in obtaining suitable counsel.The Court of Appeals af®rmed the fee award.
Held: The fee-shifting statutes at issue do not permit enhancement of a
fee award beyond the lodestar amount to re¯ect the fact that a party'sattorneys were retain

In [80]:
search_word = 'dissenting'
matches = df[df['text'].str.lower().str.contains(search_word.lower(), na=False)]

if not matches.empty:
    print(f"\nFound {len(matches)} matches for '{search_word}':")
    for index, row in matches.iterrows():
        text_lower = row['text'].lower()
        idx = text_lower.find(search_word.lower())
        if idx != -1:
            start = max(idx - 50, 0)
            end = min(idx + len(search_word) + 50, len(row['text']))
            snippet = row['text'][start:end].replace('\n', ' ')
            print(f"  Row {index}: ...{snippet}... [link: {row['url']}]")
        else:
            print(f"  Row {index}: {row['text'][:100]}... [link: {row['url']}]")
else:
    print(f"\nNo matches found for '{search_word}' in the text column.")



Found 1679 matches for 'dissenting':
  Row 1: ...nor, and Kennedy, JJ., joined. Scalia, J., ®led a dissenting opinion, post, p. 296. Thomas, J., took no part i... [link: https://supreme.justia.com/cases/federal/us/502/279/case.pdf]
  Row 2: ...nor, and Kennedy, JJ., joined. Scalia, J., ®led a dissenting opinion, in which Souter, J., joined, post, p. 42... [link: https://supreme.justia.com/cases/federal/us/502/410/case.pdf]
  Row 4: ... Maryland v.Craig, 497 U. S. 836, 864±865 (1990) (dissenting opinion). The dif®culty with the Wigmore-Harlan v... [link: https://supreme.justia.com/cases/federal/us/502/346/case.pdf]
  Row 11: ...ordered. Justice Blackmun, concurring in part and dissenting in part. I have wandered the maze of Indian statu... [link: https://supreme.justia.com/cases/federal/us/502/251/case.pdf]
  Row 12: ...nor, J., ®led an opin- ion concurring in part and dissenting in part, in which Stevens, J., joined, post, p. 7... [link: https://supreme.justia.com/cases/federal/us/502

In [81]:
import re

# Pattern: e.g., "Scalia, J., dissenting" at the end of a line or followed by a line break
# Only match if '., dissenting' is at the end of a line or followed by a line break
# Handles both Unix (\n) and Windows (\r\n) line endings
dissent_pattern = re.compile(r"([A-Z][a-zA-Z’\.\-]+(?:,? (?:J\.|JJ\.|C\.J\.))?(?:,? and [A-Z][a-zA-Z’\.\-]+,? (?:J\.|JJ\.|C\.J\.))?), dissenting(?=\s*$|\r?\n)", re.IGNORECASE|re.MULTILINE)

print("Concise dissenting section headers found in the text column:")
for idx, row in df.iterrows():
    matches = dissent_pattern.findall(row['text'])
    for match in matches:
        print(f"Row {idx}: {match.strip()}, dissenting [link: {row['url']}]" )

Concise dissenting section headers found in the text column:
Row 1: Scalia, J., dissenting [link: https://supreme.justia.com/cases/federal/us/502/279/case.pdf]
Row 1: Scalia, J., dissenting [link: https://supreme.justia.com/cases/federal/us/502/279/case.pdf]
Row 1: Scalia, J., dissenting [link: https://supreme.justia.com/cases/federal/us/502/279/case.pdf]
Row 1: Scalia, J., dissenting [link: https://supreme.justia.com/cases/federal/us/502/279/case.pdf]
Row 1: Scalia, J., dissenting [link: https://supreme.justia.com/cases/federal/us/502/279/case.pdf]
Row 2: Scalia, J., dissenting [link: https://supreme.justia.com/cases/federal/us/502/410/case.pdf]
Row 2: Scalia, J., dissenting [link: https://supreme.justia.com/cases/federal/us/502/410/case.pdf]
Row 2: Scalia, J., dissenting [link: https://supreme.justia.com/cases/federal/us/502/410/case.pdf]
Row 2: Scalia, J., dissenting [link: https://supreme.justia.com/cases/federal/us/502/410/case.pdf]
Row 2: Scalia, J., dissenting [link: https://sup

In [82]:
# Extract and print 'Opinion of JUSTICE NAME' section headers
opinion_pattern = re.compile(r"Opinion of ([A-Z][a-zA-Z’\.\-]+,? (?:J\.|JJ\.|C\.J\.))", re.IGNORECASE)

print("Opinion section headers found in the text column:")
for idx, row in df.iterrows():
    matches = opinion_pattern.findall(row['text'])
    for match in matches:
        print(f"Row {idx}: Opinion of {match.strip()} [link: {row['url']}]" )

Opinion section headers found in the text column:
Row 4: Opinion of Thomas, J. [link: https://supreme.justia.com/cases/federal/us/502/346/case.pdf]
Row 4: Opinion of Thomas, J. [link: https://supreme.justia.com/cases/federal/us/502/346/case.pdf]
Row 4: Opinion of Thomas, J. [link: https://supreme.justia.com/cases/federal/us/502/346/case.pdf]
Row 4: Opinion of Thomas, J. [link: https://supreme.justia.com/cases/federal/us/502/346/case.pdf]
Row 4: Opinion of Thomas, J. [link: https://supreme.justia.com/cases/federal/us/502/346/case.pdf]
Row 4: Opinion of Thomas, J. [link: https://supreme.justia.com/cases/federal/us/502/346/case.pdf]
Row 4: Opinion of Thomas, J. [link: https://supreme.justia.com/cases/federal/us/502/346/case.pdf]
Row 4: Opinion of Thomas, J. [link: https://supreme.justia.com/cases/federal/us/502/346/case.pdf]
Row 4: Opinion of Thomas, J. [link: https://supreme.justia.com/cases/federal/us/502/346/case.pdf]
Row 10: Opinion of Scalia, J. [link: https://supreme.justia.com/case

In [139]:
# Extract and print 'Opinion of JUSTICE NAME' section headers
opinion_pattern = re.compile(r"Concurrance([A-Z][a-zA-Z’\.\-]+,? (?:J\.|JJ\.|C\.J\.))", re.IGNORECASE)

print("Opinion section headers found in the text column:")
for idx, row in df.iterrows():
    matches = opinion_pattern.findall(row['text'])
    for match in matches:
        print(f"Row {idx}: Opinion of {match.strip()} [link: {row['url']}]" )

Opinion section headers found in the text column:


## Looking at Textual string 

- for hits on textualism

In [140]:
# find any string hit for 'textualism' and print the row number & pdf url
search_word = 'textual'
matches = df[df['text'].str.lower().str.contains(search_word.lower(), na=False)]
if not matches.empty:
    print(f"\nFound {len(matches)} matches for '{search_word}':")
    for index, row in matches.iterrows():
        text_lower = row['text'].lower()
        idx = text_lower.find(search_word.lower())
        if idx != -1:
            start = max(idx - 50, 0)
            end = min(idx + len(search_word) + 50, len(row['text']))
            snippet = row['text'][start:end].replace('\n', ' ')
            print(f"  Row {index}: ...{snippet}... [link: {row['url']}]")
        else:
            print(f"  Row {index}: {row['text'][:100]}... [link: {row['url']}]")




Found 548 matches for 'textual':
  Row 2: ...ctment. The Court makes no attempt to establish a textual or structural basis for overriding the plain mean... [link: https://supreme.justia.com/cases/federal/us/502/410/case.pdf]
  Row 5: ...spital's arguments become pallid in the light of atextual difference far more glaring than any of them: whi... [link: https://supreme.justia.com/cases/federal/us/502/215/case.pdf]
  Row 22: ...suits brought by private parties, and that the nontextual elements of theCommerce Clause have not gone unen... [link: https://supreme.justia.com/cases/federal/us/502/437/case.pdf]
  Row 28: ...t to Parts II±B and II±C, concluding that: 1. The textual evolution of § 5037(c)(1)(B) and the relevant leg... [link: https://supreme.justia.com/cases/federal/us/503/291/case.pdf]
  Row 29: ...Compromise was enacted.Pp. 437±439. (b) The three textual features of subsection (b)(6)(A) cited by re- spo... [link: https://supreme.justia.com/cases/federal/us/503/429/case.pdf]
  R

In [141]:
# create a search function which takes input of a string and searches the text column
import re

def search_text(df, search_word):
    total_matches = 0
    matches = df[df['text'].str.lower().str.contains(search_word.lower(), na=False)]
    for index, row in matches.iterrows():
        text = row['text']
        for m in re.finditer(re.escape(search_word), text, re.IGNORECASE):
            total_matches += 1
            start = max(m.start() - 50, 0)
            end = min(m.end() + 50, len(text))
            snippet = text[start:end].replace('\n', ' ')
            print(f"  Row {index}: ...{snippet}... [{row['url']}] (pos {m.start()}-{m.end()})")
    if total_matches > 0:
        print(f"\nFound {total_matches} total matches for '{search_word}'.")
    else:
        print(f"\nNo matches found for '{search_word}' in the text column.")

In [142]:
search_text(df, 'textualism')

  Row 194: ...hat our rejection of it constitutes a triumph of ªtextualismº over ªcommon sense,º post, at 146, and the resul... [https://supreme.justia.com/cases/federal/us/508/129/case.pdf] (pos 12032-12042)
  Row 194: ...1968, whenthe statute was enacted, and 1987, when textualism replacedcommon sense in its interpretation, the b... [https://supreme.justia.com/cases/federal/us/508/129/case.pdf] (pos 38235-38245)
  Row 813: ...rpose isat stake,Justice Scalia’ s cocktail-party textualism, post, at 718, must yield to the Congress of the ... [https://supreme.justia.com/cases/federal/us/529/694/case.pdf] (pos 31460-31470)
  Row 1938: ...essee , 1  Wheat. 304, 328–330 (1816); Amar, Intratextualism, 112Harv. L. Rev. 747 (1999).  The Constitution i... [https://supreme.justia.com/cases/federal/us/576/787/case.pdf] (pos 97200-97210)
  Row 2127: .... Co. , 269 U. S. 385, 391 (1926); see also Note, Textualism as Fair Notice, 123 Harv. L. Rev. 542,  543 (2009... [https://supreme.justia.com/cases/

In [176]:
search_text(df, 't-of-te')

  Row 2432: ...-sent appears to dismiss the doctrine as a “get-out-of-text free car[d].” Ibid. The dissent even seems to s... [https://supreme.justia.com/cases/federal/us/597/20-1530/case.pdf] (pos 118853-118860)
  Row 2432: ...r questions doctri ne” magically appear as get- out-of-text-free cards. 8  Today, one of those broader goal... [https://supreme.justia.com/cases/federal/us/597/20-1530/case.pdf] (pos 182764-182771)
  Row 2458: ... on special  canons “magically appearing as get-out-of-text-free cards”                      6 SACKETT v. E... [https://supreme.justia.com/cases/federal/us/598/21-454/case.pdf] (pos 141034-141041)
  Row 2485: ...ajor questions doctrine’ magically appearas get-out-of-text-free cards”).  And I grant that some artic - ul... [https://supreme.justia.com/cases/federal/us/600/22-506/case.pdf] (pos 68210-68217)

Found 4 total matches for 't-of-te'.


In [170]:
search_text(df, 'major questions doctrine')

  Row 2292: ...468 (2001).  And it would also  conflict with the major questions doctrine, which is based on the expectation that Congress ... [https://supreme.justia.com/cases/federal/us/591/18-587/case.pdf] (pos 110527-110551)
  Row 2385: ...consequential agency  rules have run afoul of the major questions doctrine.  E.g.,  MCI Telecommunications Corp. v. American... [https://supreme.justia.com/cases/federal/us/595/21a244/case.pdf] (pos 21338-21362)
  Row 2385: ...arks omitted).      The Court rightly applies the major questions doctrine and concludes that this lone statutory subsection... [https://supreme.justia.com/cases/federal/us/595/21a244/case.pdf] (pos 22276-22300)
  Row 2385: .... We have nothing like that here. *  Why does the major questions doctrine matter?  It en- sures that the national governmen... [https://supreme.justia.com/cases/federal/us/595/21a244/case.pdf] (pos 23902-23926)
  Row 2385: ...Found- ing, 130 Yale L. J. 1490, 1502 (2021). The major questions doctrine 

In [169]:
# trying to get all hits for major questions doctrine
search_text(df, 'jor q')

  Row 810: ... more likelyto have focused upon, and answered, major questions, whileleaving interstitial matters to ans... [https://supreme.justia.com/cases/federal/us/529/120/case.pdf] (pos 88169-88174)
  Row 2232: ...  different names.  Consider, for example, the “major questions” doctrine. Under our precedents, an agenc... [https://supreme.justia.com/cases/federal/us/588/17-6086/case.pdf] (pos 88899-88904)
  Row 2232: ...canon of statutory  construction, we apply the ma jor questions doctrine in service of the constitutional... [https://supreme.justia.com/cases/federal/us/588/17-6086/case.pdf] (pos 89991-89996)
  Row 2292: ...8 (2001).  And it would also  conflict with the major questions doctrine, which is based on the expectati... [https://supreme.justia.com/cases/federal/us/591/18-587/case.pdf] (pos 110529-110534)
  Row 2385: ...n marks omitted).  We  sometimes call this the ma jor questions doctrine.  Gundy v.  United States , 588 ... [https://supreme.justia.com/cases/federal/us

In [145]:
search_text(df, 'chevron')

  Row 21: ...ommercial Workers, 484 U. S. 112, 123 (1987); cf. Chevron U. S. A. Inc. v.Natural Resources Defense Council... [https://supreme.justia.com/cases/federal/us/502/527/case.pdf] (pos 21278-21285)
  Row 21: ...f employees and nonemploy-ees, we were saying, in Chevron terms, that § 7 speaks to the issue of nonemploye... [https://supreme.justia.com/cases/federal/us/502/527/case.pdf] (pos 22869-22876)
  Row 21: ...e agencycharged with administering a statute. See Chevron U. S. A. Inc. v.Natural Resources Defense Council... [https://supreme.justia.com/cases/federal/us/502/527/case.pdf] (pos 40230-40237)
  Row 21: ...ority today, like that of the Court of Appeals in Chevron, is to adopt a static judi- cial construction of ... [https://supreme.justia.com/cases/federal/us/502/527/case.pdf] (pos 45091-45098)
  Row 25: ...rules gov-erning voting: It does not. See, e. g., Chevron U. S. A. Inc. v.Natural Resources Defense Council... [https://supreme.justia.com/cases/federal/us/502/491/cas

In [177]:
from bs4 import BeautifulSoup

def parse_html_metadata(html):
    soup = BeautifulSoup(html, 'html.parser')
    data = {}
    for div in soup.find_all('div', class_='flex-col'):
        label = div.find('strong')
        value = div.find('span')
        if label and value:
            key = label.text.strip().replace(':', '')
            val = value.text.strip()
            data[key] = val
    return data

# Parse all html_metadata and create new columns in the DataFrame
parsed = df['html_metadata'].apply(parse_html_metadata)
parsed_df = parsed.apply(pd.Series)
df_with_metadata = pd.concat([df, parsed_df], axis=1)

# Show the new columns for a sample row
row_index = 2400
print(df_with_metadata.loc[row_index, ['Docket No.', 'Granted', 'Argued', 'Decided']])

Docket No.              20-979
Granted          June 28, 2021
Argued        December 6, 2021
Decided           May 16, 2022
Name: 2400, dtype: object


In [178]:
df_with_metadata.columns

Index(['id', 'text', 'url', 'timestamp', 'html_metadata', 'Argued', 'Decided',
       'Granted', 'Reargued', 'Question certified',
       'Certified question to Arizona Supreme Court', 'Docket No.',
       'Juris Postponed'],
      dtype='object')

In [179]:
df_with_metadata.Argued

0        December 2, 1991
1         October 7, 1991
2        October 15, 1991
3        October 15, 1991
4        November 5, 1991
              ...        
2599       March 31, 2025
2600       April 21, 2025
2601        March 3, 2025
2602    December 10, 2024
2603     January 22, 2025
Name: Argued, Length: 2604, dtype: object

In [180]:
df_with_metadata.Decided

0       January 14, 1992
1       January 14, 1992
2       January 15, 1992
3       November 5, 1991
4       January 15, 1992
              ...       
2599        June 5, 2025
2600       June 12, 2025
2601        June 5, 2025
2602        May 29, 2025
2603        May 15, 2025
Name: Decided, Length: 2604, dtype: object

In [181]:
df_with_metadata['Question certified'].value_counts()

Question certified
November 30, 1999    1
Name: count, dtype: int64

In [182]:
HTML(html_content) # Convert HTML to string if needed

In [183]:
df_with_metadata['Certified question to Arizona Supreme Court'].value_counts()

Certified question to Arizona Supreme Court
December 12, 2001    1
Name: count, dtype: int64

In [184]:
df_with_metadata['Juris Postponed'].value_counts()

Juris Postponed
June 19, 2017    1
Name: count, dtype: int64

In [185]:
df_with_metadata['Docket No.']

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
2599     24-154
2600     24-275
2601    23-1259
2602     23-975
2603    23-1239
Name: Docket No., Length: 2604, dtype: object